In [ ]:
!pip install muspy
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 4.7 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591954 sha256=f6a74c2a2db12576e147b5886a8d1ab6b26b3a97827be928d1b1f5bd72d96a89
  Stored in directory: /root/.cache/pip/wheels/2a/5a/e3/30eeb9a99350f3f7e21258fcb132743eef1a4f49b3505e76b6
Successfully built pretty-midi
  Attempting uninstall: music21
    Found existing installation: music21 5.5.0
    Uninstalling music21-5.5.0:
      Successfully uninstalled music21-5.5.0
Loo

In [ ]:
import muspy as mp
import music21
import os

import random
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def create_empty_list(n : int):
  dict_note = []
  for i in range(0,n):
    dict_note.append(0.0)
  return dict_note

def normalize_list(l : list):
  total = sum(l)
  if total > 0.0:
    for i in range(len(l)):
      l[i] = l[i]/total

def randomize(l : list):
  for i in range(len(l)):
    l[i] += random.randint(0,5)

Transition matrix : KS x KS  ->  list_ks

Generation matrix : N x KS  ->  list_notes

Duration Generation matrix : dur x N  ->  list_notes_dur


In [ ]:
list_ks = []
list_notes = []
list_notes_dur = []

for i in range(0,12): #on crée une liste pour chaque note qui servira à mettre la fréquence de ses notes voisines
  prob_allowed_ks = create_empty_list(24)
  list_notes.append(prob_allowed_ks)
  
  prob_duration_notes = create_empty_list(60)
  list_notes_dur.append(prob_duration_notes)

for i in range(0,24):
  prob_ks = create_empty_list(24)
  list_ks.append(prob_ks)

On va générer des notes à partir des KS, puis avec les observations générées, nous allons générer leurs durée.

In [ ]:
data_dir = pathlib.Path("major_parts")
filenames = glob.glob(str(data_dir/'*.mid*'))
print('Number of files:', len(filenames))

In [ ]:
sample_file = filenames[20]
pm = pretty_midi.PrettyMIDI(sample_file)

In [ ]:
directory = "parts"
pbar = ProgressBar()

for filename in pbar(os.listdir(directory)):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      music = mp.from_music21(music21.converter.parse(f))  
      key_signatures = music.key_signatures
      notes = music.tracks[0].notes

      i_ks = 0

      for i in range(0, len(notes)-1):
        if notes[i].time != notes[i+1].time:    
          list_ks_for_note = list_notes[notes[i].pitch%12] #on récupère la liste des KS possibles de la note courante


          if i_ks == len(key_signatures)-1:  #si c'est le dernier on prendra une plage entre le temps de départ du dernier et la fin du morceau
            ks_ind = key_signatures[i_ks].root + 12 if key_signatures[i_ks].mode == "minor" else key_signatures[i_ks].root
            list_ks_for_note[ks_ind] += 1

          else:#sinon on prendra la plage entre deux KeySignature
            if notes[i].time >= key_signatures[i_ks].time and notes[i].time < key_signatures[i_ks + 1].time: #si la note est dans la i_ks ème KS
              ks_ind = key_signatures[i_ks].root + 12 if key_signatures[i_ks].mode == "minor" else key_signatures[i_ks].root
              list_ks_for_note[ks_ind] += 1
            else:                                                                                            #si elle est dans la i_ks+1 ème KS
              ks_ind = key_signatures[i_ks + 1].root + 12 if key_signatures[i_ks].mode == "minor" else key_signatures[i_ks + 1].root
              list_ks_for_note[ks_ind] += 1

              #MAJ du voisinage des KS pour les transitions
              curr_ks = key_signatures[i_ks].root + 12 if key_signatures[i_ks].mode == "minor" else key_signatures[i_ks].root
              list_ks[curr_ks][ks_ind] += 1
              list_ks[ks_ind][curr_ks] += 1


              i_ks += 1

        list_to_update_dur = list_notes_dur[notes[i].pitch] #on récupère la liste des vélocités de la note courante
        list_to_update_dur[notes[i].duration] += 1

      list_to_update_dur = list_notes_dur[notes[len(notes)-1].pitch] 
      list_to_update_dur[notes[len(notes)-1].duration] += 1

IndexError: ignored

In [ ]:
for i in range(len(list_notes)):
  normalize_list(list_notes[i])
  normalize_list(list_notes_dur[i])

for i in range(len(list_ks)):
  normalize_list(list_ks[i])

In [ ]:
for e in list_notes_dur:
  print(e)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

## Génération par HMM

Génération des notes

In [ ]:
A = torch.tensor(list_ks)
A

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6667, 0.0000, 0.0000, 0.3333, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000

In [ ]:
AB = torch.tensor(list_notes)
AB

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
L, V = torch.linalg.eig(A)
pi0 = V[:,0]
pi0 = pi0/torch.sum(pi0)
pi0 = torch.real(pi0)

In [ ]:
N=900 # taille de la séquence que l'on veut
seq_notes = [] #séquence des notes
# initialisation
current_state = 2
obs = torch.multinomial(AB[:,current_state],1).item()
seq_notes.append(obs)

seq_KS = [] #séquence des KS 
seq_KS.append(current_state)

for i in range(0,N):
  # transition pour chaque partie (A, B, C)
  if (i+1)%(N*3+1) == 0:
    print(i, "change KS")
    current_state = torch.multinomial(A[:,current_state],1).item()
    seq_KS.append(current_state)
    seq_notes.append(0) #on génère une obs de 0 à chaque changement pour que la génération crée une pause
  
  #emission
  obs = torch.multinomial(AB[:,current_state],1).item()
  seq_notes.append(obs)

# optionnel
print(seq_KS)
print(seq_notes)

[2]
[68, 78, 68, 68, 66, 76, 71, 66, 76, 73, 73, 65, 81, 64, 62, 66, 68, 67, 62, 62, 73, 79, 68, 78, 68, 68, 75, 71, 78, 69, 66, 81, 68, 79, 66, 65, 78, 64, 67, 78, 67, 64, 70, 75, 68, 65, 78, 68, 76, 74, 65, 68, 71, 69, 79, 64, 76, 66, 67, 76, 74, 67, 62, 79, 62, 74, 62, 66, 75, 66, 64, 71, 71, 65, 65, 64, 66, 70, 78, 65, 71, 64, 71, 71, 78, 76, 74, 74, 66, 78, 70, 73, 70, 69, 68, 62, 66, 78, 79, 74, 71, 67, 67, 62, 73, 78, 75, 67, 68, 78, 68, 66, 71, 78, 70, 78, 78, 79, 62, 68, 79, 79, 73, 69, 75, 79, 62, 72, 67, 78, 66, 71, 64, 65, 68, 70, 70, 66, 76, 78, 65, 67, 74, 67, 73, 78, 78, 70, 69, 81, 66, 66, 78, 79, 76, 64, 66, 66, 74, 64, 71, 71, 75, 69, 66, 65, 66, 66, 68, 69, 70, 64, 64, 75, 62, 64, 62, 72, 79, 68, 79, 64, 62, 64, 78, 78, 75, 64, 64, 79, 68, 65, 69, 79, 73, 81, 69, 66, 70, 75, 78, 68, 71, 74, 74, 71, 66, 69, 66, 62, 76, 79, 67, 78, 79, 67, 67, 75, 71, 71, 79, 70, 75, 68, 72, 65, 68, 78, 67, 62, 62, 78, 71, 71, 67, 68, 81, 69, 69, 76, 69, 67, 64, 69, 65, 66, 81, 71, 62,

Génération de la vélocité

In [ ]:
def compensate_zeros(l : list):
  l[random.randint(0,len(l)-1)] = 1.0

In [ ]:
compensate_zeros(list_notes_dur[0])
B = torch.tensor(numpy.transpose(list_notes_dur))
B

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [ ]:
seq_dur = []

for i in range(len(seq_notes)):
  etat = seq_notes[i]
  obs = torch.multinomial(B[:,etat],1).item()
  seq_dur.append(obs)

print("Séquence d'observations générée :",seq_dur)

Séquence d'observations générée : [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 24, 6, 6, 6, 6, 24, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 30, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 12, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 18, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 30, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 18, 6, 6, 6, 6, 6, 6, 12, 30, 6, 6, 6, 6, 24, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 24, 6, 12, 6, 6, 6, 6, 18, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 24, 6, 6, 6, 6, 12, 6, 6, 18, 6, 6, 6, 6, 6, 6, 6, 6, 6, 

In [ ]:
generated_notes = []
prev_start = 0
prev_dur = 0
for i in range(len(notes_index)):
  pitch = seq_notes[i] + 72
  duration = 0.05 * seq_dur[i]
  step = duration
  start = prev_start + step
  end = start + duration
  input_note = (pitch, step, duration)
  generated_notes.append((*input_note, start, end))
  prev_start = start

generated_notes = pd.DataFrame(
    generated_notes, columns=('pitch','step','duration', 'start', 'end'))

print(generated_notes.head(10))

array([[    0,    68,     8,    54],
       [    8,    78,     9,    40],
       [   15,    68,     6,    41],
       ...,
       [11012,    75,     6,    41],
       [11026,    68,     7,    45],
       [11037,    70,    15,    50]])

In [ ]:
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

In [ ]:
music_hmm.show('midi')

In [ ]:
music_hmm.write("midi", fp="./content.midi")

'./content.midi'